In [2]:
import glob

import xarray as xr

import pyremo as pr

## Monthly

In [2]:
files = glob.glob("/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e*m*.nc")
files.sort()
files

['/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200001.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200002.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200003.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200004.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200005.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200006.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200007.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200008.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200009.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200010.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200011.nc',
 '/work/ch0636/g300046/from_Mistral/remo_results_056000/2000/e056000m200012.nc']

In [3]:
import xarray as xr

ds = xr.open_mfdataset(files, data_vars="minimal", coords="minimal")

In [8]:
ds = ds.copy()
da = ds.TEMP2.to_dataset()
da = da.merge(ds.rotated_latitude_longitude)
da = pr.parse_dates(da, use_cftime=True)

In [7]:
da.TEMP2.encoding["_FillValue"] = 1.0e20

In [8]:
da.to_netcdf("remo_EUR-11_TEMP2_2000.nc")

## Hourly

In [9]:
from dask.distributed import Client

client = Client()

In [8]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /user/g300046/levante-spawner-advanced//proxy/39483/status,
Dashboard: /user/g300046/levante-spawner-advanced//proxy/39483/status,Workers: 8
Total threads: 48,Total memory: 58.59 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38229,Workers: 8
Dashboard: /user/g300046/levante-spawner-advanced//proxy/39483/status,Total threads: 48
Started: Just now,Total memory: 58.59 GiB
Comm: tcp://127.0.0.1:46667,Total threads: 6
Dashboard: /user/g300046/levante-spawner-advanced//proxy/39553/status,Memory: 7.32 GiB
Nanny: tcp://127.0.0.1:37791,


In [3]:
from pyremo.archive import RemoArchive

archive = RemoArchive("/work/ch0636/g300046/from_Mistral/remo_results_056000")

100%|██████████| 86/86 [00:00<00:00, 3161.38it/s]


In [4]:
files = archive._extract_code(167, time_range=("1999-12", "2000-01"), parallel=True)

100%|██████████| 2/2 [00:00<00:00, 1573.85it/s]


In [5]:
files

['/scratch/g/g300046/_archive_extract/e056000e_c167_199912.nc',
 '/scratch/g/g300046/_archive_extract/e056000e_c167_200001.nc']

In [6]:
ds = xr.open_mfdataset(files, preprocess=pr.parse_dates, data_vars="minimal")
ds

<xarray.Dataset>
Dimensions:                     (rlon: 433, rlat: 433, height2m: 1, time: 1488)
Coordinates:
  * rlon                        (rlon) float64 -28.93 -28.82 ... 18.49 18.6
  * rlat                        (rlat) float64 -23.93 -23.82 ... 23.49 23.6
  * height2m                    (height2m) float64 2.0
    lon                         (rlat, rlon) float64 dask.array<chunksize=(433, 433), meta=np.ndarray>
    lat                         (rlat, rlon) float64 dask.array<chunksize=(433, 433), meta=np.ndarray>
  * time                        (time) datetime64[ns] 1999-12-01T01:00:00 ......
Data variables:
    rotated_latitude_longitude  |S1 b''
    TEMP2                       (time, height2m, rlat, rlon) float32 dask.array<chunksize=(744, 1, 433, 433), meta=np.ndarray>
Attributes:
    remo_version:  2.0.0
    git_branch:    nc_meta
    git_hash:      c4ee7f4
    system:        Linux eddy3 2.6.32-754.33.1.el6.x86_64 #1 SMP Mon Aug 10 1...

In [7]:
ds.TEMP2.encoding["_FillValue"] = 1.0e20
for c in ds.coords:
    ds[c].encoding["_FillValue"] = None

In [8]:
ds.sel(time=slice("2000-01-01T00:00:00", "2000-01-03T00:00:00")).to_netcdf(
    "remo_EUR-11_TEMP2_1hr.nc"
)